In [44]:
os.splitext("test.txt")

AttributeError: module 'os' has no attribute 'splitext'

In [52]:
import os

from mutagen.flac import FLAC
from mutagen.mp3 import MP3
from mutagen.easyid3 import EasyID3
from mutagen.wave import WAVE

from shutil import copyfile,move


from tqdm import tqdm

from functools import partial

def get_new_path(fpath,processor,ext=None):
    if ext is None:
        ext = os.path.splitext(fpath)[-1]
    fields = ["artist","album","title"]
    print(processor)
    audio = processor(fpath)
    song_attrs = {}
    for field in fields:
        field_val = f"Unknown {field}"
        if not isinstance(audio,dict):
            try:
                val_poss = getattr(audio,field)
                if val_poss is not None:
                    field_val = getattr(audio,field)
            except:
                pass
        elif field in audio:
            field_vals = audio[field]
            if len(field_vals)>0:
                field_val = field_vals[0]

        song_attrs[field] = field_val
    if song_attrs["title"] == f"Unknown title":
        song_attrs["title"] = os.path.splitext(os.path.split(fpath)[-1])[0]

    return os.path.join("/music_store/holding/soulseek",song_attrs["artist"],song_attrs["album"],song_attrs["title"]+ext)

def main(_):
    # get files
    print("Getting files...")
    exts = {".flac":FLAC,".mp3":partial(MP3,ID3=EasyID3)}
    fpaths = {ext:[] for ext in exts}
    # get filepaths
    for root, dirs, files in os.walk("/soulseek/"):
        for filename in files:
            ext = None
            for ext in exts:
                if filename.endswith(ext):
                    fpaths[ext].append(os.path.join(root,filename))
                    break
    print("Found files with the following extensions:\n","\n".join([ext + ":" + str(len(fpaths[ext])) + '\n' for ext in fpaths]))
                    
    print("Creating mappings to new destinations...")
    fpath_mappings = {}
    for ext in fpaths:
        for fpath in tqdm(fpaths[ext]):
            fpath_mappings[fpath] = get_new_path(fpath,exts[ext])

    print("Moving files over...")
    counter = 0
    for fpath in tqdm(fpath_mappings):
        dest = fpath_mappings[fpath]
        # check haven't already imported
        if not os.path.exists(dest):
            folder,file = os.path.split(dest)
            if not os.path.exists(folder):
                os.makedirs(folder)
            copyfile(fpath,dest)
            counter +=1
    
    print("File import complete.",f"Imported {counter} out of {len(fpath_mappings)} files")

In [106]:
main(None)

Getting files...
Found files with the following extensions:
 .flac:1

.mp3:1

Creating mappings to new destinations...


100%|█████████████████| 1/1 [00:00<00:00, 167.61it/s]


Moving files over...


100%|██████████████████| 2/2 [00:01<00:00,  1.29it/s]

File import complete. Imported 2 out of 2 files


In [31]:
exts = {".flac":FLAC,".mp3":partial(MP3,ID3=EasyID3),".wav":TinyTag.get}
fpaths = {ext:[] for ext in exts}
# get filepaths
for root, dirs, files in os.walk("/soulseek/"):
    for filename in files:
        ext = None
        for ext in exts:
            if filename.endswith(ext):
                fpaths[ext].append(os.path.join(root,filename))
                break

In [53]:
fpath_mappings = {}
for ext in fpaths:
    for fpath in tqdm(fpaths[ext]):
        fpath_mappings[fpath] = get_new_path(fpath,exts[ext])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 103.72it/s]


<class 'mutagen.flac.FLAC'>


0it [00:00, ?it/s]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 190.74it/s]

<bound method TinyTag.get of <class 'tinytag.tinytag.TinyTag'>>


In [54]:
fpath_mappings

{"/soulseek/cutpaperglue/SOPHIE - Oil of Every Pearl's Un-Insides/08 - Immaterial.flac": '/music_store/holding/soulseek/Unknown artist/Unknown album/08 - Immaterial.flac',
 '/soulseek/erickt/Mix/Deeperlove - Renegade Master (Volac Remix).wav': '/music_store/holding/soulseek/Deeperlove/Unknown album/Deeperlove - Renegade Master (Volac Remix).wav'}

In [5]:
fpath_mappings

{"/soulseek/cutpaperglue/SOPHIE - Oil of Every Pearl's Un-Insides/08 - Immaterial.flac": "/music_store/holding/soulseek/SOPHIE/Oil of Every Pearl's Un-Insides/Immaterial.flac",
 '/soulseek/erickt/Mix/Deeperlove - Renegade Master (Volac Remix).wav': '/music_store/holding/soulseek/Unknown artist/Unknown album/Unknown title.wav'}

In [8]:
pip install tinytag

  Created wheel for tinytag: filename=tinytag-1.6.0-py3-none-any.whl size=32473 sha256=74097b5dc82ee3a92d30137c771329a6e3c2edc7baa8efc94c0eb08901db80f7
  Stored in directory: /root/.cache/pip/wheels/65/21/c2/0c78b7b114e97307f6429f903ff8776aa200aa8eff9b1c5edc
Successfully built tinytag
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [10]:
from tinytag import TinyTag

test = WAVE("/soulseek/erickt/Mix/Deeperlove - Renegade Master (Volac Remix).wav")

In [17]:
test

{}

In [15]:
tag = TinyTag.get("/soulseek/erickt/Mix/Deeperlove - Renegade Master (Volac Remix).wav")

In [21]:
getattr(tag, "album")

In [7]:
test

{}